<a href="https://colab.research.google.com/github/wolfinwallst/Machine_Learning_Deep_basis/blob/main/RL_A2C_portfolio_optimization_PoC_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
import datetime

import gym
from gym import spaces

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
##############################################
# 1. 데이터 다운로드 및 전처리
##############################################
# S&P500 상위 20종목 (예시 티커)
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "TSLA", "NVDA", "BRK-B",
           "JNJ", "UNH", "V", "MA", "HD", "PG", "JPM", "BAC", "VZ", "DIS", "PFE", "XOM"]

# 10년 전부터 오늘까지의 데이터
end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.datetime.now() - datetime.timedelta(days=365*10)).strftime('%Y-%m-%d')

data = yf.download(tickers, start=start_date, end=end_date)["Close"]
data.dropna(inplace=True)

# 일별 수익률 계산
returns_df = data.pct_change().dropna()

[*********************100%***********************]  8 of 8 completed
ERROR:yfinance:
8 Failed downloads:
ERROR:yfinance:['AAPL', 'BRK-B', 'META', 'MSFT', 'NVDA', 'GOOGL', 'AMZN', 'TSLA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [ ]:
##############################################
# 2. 벤치마크: 시가총액 비중 배분 계산
##############################################
market_caps = []
for ticker in tickers:
    try:
        info = yf.Ticker(ticker).info
        cap = info.get("marketCap", None)
        # marketCap 정보가 없으면 임의의 값 할당
        if cap is None:
            cap = 1e9
        market_caps.append(cap)
    except Exception as e:
        market_caps.append(1e9)

market_caps = np.array(market_caps)
benchmark_weights = market_caps / market_caps.sum()
print("Benchmark weights (시가총액 비중):")
print(dict(zip(tickers, np.round(benchmark_weights, 4))))

# 벤치마크의 일별 수익률 = 각 종목의 일별 수익률에 시가총액 비중 곱한 값의 합산
benchmark_daily_returns = returns_df.dot(benchmark_weights)

In [ ]:
##############################################
# 3. 환경(Environment) 정의: HistoricalPortfolioEnv
##############################################
# Gym의 Env 클래스를 상속받아, 과거 수익률 데이터를 순차적으로 제공하는 포트폴리오 환경을 정의
class HistoricalPortfolioEnv(gym.Env):
    """
    과거 일별 수익률 데이터를 순차적으로 제공하는 환경.
    - 상태(state): 해당일의 각 종목 수익률 (vector)
    - 행동(action): 각 종목에 할당할 투자비중 (합=1)
    - 보상(reward): 선택한 투자비중과 실제 수익률의 내적 (포트폴리오 당일 수익률)
    """
    def __init__(self, returns):
        super(HistoricalPortfolioEnv, self).__init__()
        self.returns = returns  # numpy array, shape=(T, n_assets)
        self.n_assets = returns.shape[1]
        self.current_step = 0
        # 행동 공간: 각 종목에 대한 투자 비중, 0~1 범위의 연속값 (벡터)
        self.action_space = spaces.Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)
        # 관측 공간: 각 종목의 수익률 (실수 값으로 무한대 범위)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_assets,), dtype=np.float32)

    def reset(self):
        # 환경 초기화: 시간 스텝을 0으로 초기화 후 첫 번째 수익률 벡터 반환
        self.current_step = 0
        return self.returns[self.current_step]

    def step(self, action):
        # 행동값 정규화 (합=1) & 1e-8은 분모가 0 이 되는 상황을 방지코자
        weights = action / (np.sum(action) + 1e-8)
        current_return = self.returns[self.current_step] # 현재 스텝의 종목별 수익률
        reward = np.dot(weights, current_return) # 포트폴리오 수익률
        self.current_step += 1
        done = self.current_step >= len(self.returns) # 마지막 스텝인지 확인

        if not done:
            next_state = self.returns[self.current_step]
        else:
            next_state = np.zeros(self.n_assets)
        return next_state, reward, done, {}

# 수익률 데이터를 numpy 배열로 변환 후 환경 생성
returns_np = returns_df.values
env = HistoricalPortfolioEnv(returns_np)
n_assets = env.n_assets

In [ ]:
##############################################
# 4. Actor-Critic 모델 정의 (PyTorch)
##############################################
class ActorCritic(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # Actor: Dirichlet 분포의 알파 파라미터 출력 (양수여야 하므로 softplus 후 +1)
        self.actor = nn.Linear(hidden_dim, output_dim)
        # Critic 출력층: 상태의 가치를 예측하는 선형층 (스칼라 출력)
        self.critic = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        # actor 네트웤 순서: input-fc1-relu-fc2-relu-actor
        actor_logits = self.actor(x)    # policy network 출력
        # critic 네트웤 순서: input-fc1-relu-fc2-relu-critic
        value = self.critic(x)          # state function 출력
        return actor_logits, value

input_dim = n_assets    # 입력 차원: 각 종목의 수익률 (자산 개수)
hidden_dim = 64         # 은닉층 차원
output_dim = n_assets   # 출력 차원: 각 종목에 대한 투자 비중 예측

model = ActorCritic(input_dim, hidden_dim, output_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
gamma = 0.99 # discount factor

궁금증
1) actor 네트웤 순서: `input-fc1-relu-fc2-relu-actor` 로 잡았는데
`RL_Actor_Critic_CartPole_01.ipynb` 처럼 `input-fc1-relu-fc2-softmax` 로 바꾼다면?

2) hidden_dim 을 얼마나 크게 / 작게 잡는게 좋은가?
3)

In [ ]:
##############################################
# 5. 학습 (Training) - Actor-Critic 방식
##############################################
num_episodes = 50  # 에피소드 수 (데모 목적)

for episode in range(num_episodes):
    state = env.reset()    # 환경 초기화 및 첫 상태 획득
    log_probs = []         # 각 스텝의 log-확률 저장 리스트 (Actor 업데이트용)
    values = []            # Critic의 상태 가치 예측 저장 리스트
    rewards = []           # 에피소드 동안의 보상 저장 리스트
    done = False           # 에피소드 종료 여부 플래그

    while not done:
        state_tensor = torch.FloatTensor(state).unsqueeze(0)  # [1, n_assets]
        logits, value = model(state_tensor)
        # Dirichlet 분포의 알파 파라미터 계산
        alpha = F.softplus(logits) + 1.0
        dist = torch.distributions.Dirichlet(alpha.view(-1))
        # 디리클레 분포에서 샘플링하여 행동(투자 비중) 결정
        action = dist.sample()
        log_prob = dist.log_prob(action)
        next_state, reward, done, _ = env.step(action.detach().numpy())

        log_probs.append(log_prob)
        values.append(value.squeeze(0))
        rewards.append(reward)
        state = next_state

    # 에피소드별 누적수익률 (Return) 계산 (감가율 적용)
    returns_episode = []
    R = 0
    for r in rewards[::-1]:
        R = r + gamma * R
        returns_episode.insert(0, R)
    returns_episode = torch.tensor(returns_episode, dtype=torch.float32)
    values = torch.stack(values)
    log_probs = torch.stack(log_probs)

    advantages = returns_episode - values.detach() # Advantage 계산: 실제 누적 보상과 Critic의 예측 가치 차이
    actor_loss = -(log_probs * advantages).mean() # Actor 손실: 로그 확률에 Advantage를 곱해 부호 반전 후 평균 (정책 경사법)
    critic_loss = F.mse_loss(values, returns_episode) # Critic 손실: 가치 예측과 실제 누적 보상의 평균 제곱 오차(MSE)
    loss = actor_loss + critic_loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if episode % 10 == 0:
        total_reward = sum(rewards)
        print(f"Episode {episode:3d} | Loss: {loss.item():.4f} | Total Reward: {total_reward:.4f}")

print("학습 완료")

1) 랜덤 action 을 결정할 때, Dirichlet dist.를 사용했는데 다른 방법은? 그리고 현재 상태에서 디리클레 분포를 사용해서 랜덤 action을 결정하는 방법이 얼마나 적합한가?

ChatGPT: 1. Dirichlet 분포 외의 대안
Softmax 기반 접근법 (Gumbel-Softmax 포함)

방법: 네트워크가 출력한 로짓(logit) 값에 대해 softmax 연산을 적용하면 0~1 사이의 값들이 나오고, 총합이 1이 됩니다.
장점: 구현이 간단하며, 확률 분포처럼 해석할 수 있습니다.
단점: 기본 softmax는 결정론적이어서 탐험성이 부족할 수 있으므로, Gumbel-Softmax와 같이 샘플링 기법을 도입하여 확률적 행동 선택을 유도할 수 있습니다.
Logit-Normal 분포

방법: 네트워크가 출력한 값을 가우시안 분포의 logit으로 해석하고, softmax를 통해 단순화된 분포로 변환하는 방법입니다.
장점: 연속적 확률 공간을 모델링할 수 있으며, Dirichlet와는 다른 분산 특성을 가질 수 있습니다.
단점: 파라미터 튜닝과 학습 안정성 측면에서 Dirichlet에 비해 다소 복잡할 수 있습니다.
Stick-Breaking Process

방법: 단순한 재귀적 방법으로 “막대기를 나누는” 과정(Stick-breaking)을 통해 단순체(Simplex) 상의 점을 생성합니다.
장점: 자연스럽게 단순체 제약을 만족시키며, 다양한 분포 형태를 모델링할 수 있습니다.
단점: 구현 복잡성이 증가하고, 미분가능성 문제 때문에 직접적인 학습에 사용하기 어렵습니다.
2. Dirichlet 분포 사용 적합성 평가
Dirichlet 분포가 현재 포트폴리오 배분 문제에서 랜덤 액션 결정에 적합한지를 평가할 때 고려할 기준은 다음과 같습니다.

제약 조건 만족:

판단 기준: 포트폴리오의 각 비중이 0 이상이며, 합이 1이어야 한다는 조건
판단 정도: Dirichlet 분포는 이 제약을 내재적으로 만족시키므로 매우 강한 적합성을 보입니다.
분포의 유연성 및 탐험 성질 (Entropy):

판단 기준: 네트워크가 출력하는 alpha 파라미터에 따라 분포의 집중도(Entropy)가 결정됩니다.
낮은 alpha: 분포가 넓게 퍼져(높은 entropy) 탐험성이 큼
높은 alpha: 분포가 집중되어(낮은 entropy) 결정론적으로 행동 선택
판단 정도: 적절한 alpha 값을 조절한다면, Dirichlet 분포는 탐험과 활용 사이의 균형을 잘 맞출 수 있습니다. (탐험성이 충분하면서도 제약 조건을 유지하는 점에서 중간 이상으로 적합)
학습 안정성 및 미분 가능성:

판단 기준: 정책 네트워크가 반환하는 파라미터로부터 샘플링한 행동에 대해 역전파가 원활하게 진행되어야 합니다.
판단 정도: Dirichlet 분포는 미분 가능하게 파라미터화할 수 있으므로, 학습 측면에서는 안정적이라고 볼 수 있습니다.
실제 성능 (Empirical Performance):

판단 기준: 백테스팅이나 시뮬레이션 결과를 통해, 실제 투자 성과나 누적 수익률의 안정성이 평가되어야 합니다.
판단 정도: 초기 실험 결과가 긍정적이라면, Dirichlet 분포 사용은 문제에 잘 맞는다고 판단할 수 있습니다. 다만, 하이퍼파라미터(특히 alpha 값의 초기값 및 업데이트 방식) 튜닝이 중요합니다.
결론
대안 방법: Softmax (또는 Gumbel-Softmax), Logit-Normal 분포, Stick-Breaking Process 등이 있으며, 각각의 방법은 구현 복잡성, 학습 안정성, 그리고 탐험 성향에서 차이가 있습니다.
Dirichlet 분포 평가:
적합성: 매우 높음 (특히 제약 조건 내재 만족 측면)
탐험/활용 균형: 네트워크의 alpha 파라미터 조절에 따라 결정되며, 적절하게 설정하면 중간 이상으로 적합함
판단 강도: 현재 포트폴리오 배분 문제에서 Dirichlet 분포는 강력히 추천할 수 있는 방법으로 판단되며, 실제 성능을 면밀히 관찰하면서 하이퍼파라미터 튜닝을 진행하는 것이 좋습니다.
따라서, Dirichlet 분포는 포트폴리오 배분의 제약 조건을 자연스럽게 만족시키고, 충분한 탐험성을 보장할 수 있기 때문에 적합한 선택이라 할 수 있습니다. 다만, 다른 방법들도 상황에 따라 유용할 수 있으므로, 실제 프로젝트에서는 다양한 접근법을 비교 평가하는 것이 바람직합니다.

2)

In [ ]:
##############################################
# 6. 백테스팅 (Backtesting)
##############################################
# 학습된 모델을 사용하여 전체 기간에 대해 포트폴리오 수익률 산출

# 백테스트를 위한 새로운 환경 인스턴스 생성
env_bt = HistoricalPortfolioEnv(returns_np)
state = env_bt.reset()
agent_daily_returns = []  # 에이전트가 생성한 일별 수익률 기록 리스트

while True:
    # 현재 상태를 텐서로 변환하여 모델 입력으로 사용
    state_tensor = torch.FloatTensor(state).unsqueeze(0)
    with torch.no_grad():
        # 모델을 통해 정책 네트워크의 출력 계산 (액션 결정)
        logits, _ = model(state_tensor)
        # Dirichlet 분포 파라미터 계산 (softplus 후 +1)
        alpha = F.softplus(logits) + 1.0
        dist = torch.distributions.Dirichlet(alpha.view(-1))
        # 백테스트에서는 확률 분포의 평균값을 사용하여 결정론적 정책으로 행동 결정
        weights = dist.mean
    # 선택한 투자 비중을 환경에 적용하여 다음 상태와 보상 획득
    next_state, reward, done, _ = env_bt.step(weights.numpy())
    agent_daily_returns.append(reward)
    if done:
        break
    state = next_state

# 에이전트의 일별 수익률을 numpy 배열로 변환 후 누적 수익률 계산 (초기 투자금 1 기준)
agent_daily_returns = np.array(agent_daily_returns)
agent_cumulative = np.cumprod(1 + agent_daily_returns)

# 벤치마크 누적 수익률 계산: 벤치마크 일별 수익률을 같은 기간에 맞추어 계산
benchmark_daily = benchmark_daily_returns.values[-len(agent_daily_returns):]
benchmark_cumulative = np.cumprod(1 + benchmark_daily)

# 백테스트 기간의 날짜 (원본 수익률 데이터의 인덱스 사용)
dates = returns_df.index[-len(agent_daily_returns):]

In [ ]:
##############################################
# 7. 결과 Plot: RL 에이전트 vs. Benchmark
##############################################
# 백테스팅 결과를 시각화하여 RL 에이전트와 벤치마크(시가총액 비중)의 누적 수익률 비교
plt.figure(figsize=(12,6))
plt.plot(dates, agent_cumulative, label="RL Agent Portfolio")
plt.plot(dates, benchmark_cumulative, label="Benchmark (시가총액 비중)", linestyle="--")
plt.xlabel("날짜")
plt.ylabel("누적 수익률")
plt.title("Backtesting: RL Agent vs. Benchmark")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import yfinance as yf          # 금융 데이터 다운로드를 위한 라이브러리
import numpy as np             # 수치 연산을 위한 라이브러리
import pandas as pd            # 데이터프레임 처리 라이브러리
import matplotlib.pyplot as plt# 시각화를 위한 라이브러리
import datetime                # 날짜 및 시간 계산을 위한 모듈
import gym                     # 강화학습 환경을 구축하기 위한 라이브러리
from gym import spaces         # Gym 환경의 행동 및 관측 공간 정의에 사용
import torch                   # 파이토치(PyTorch): 딥러닝 모델 구현용
import torch.nn as nn          # 신경망 계층 모듈
import torch.nn.functional as F# 활성화 함수 등 함수형 API 제공
import torch.optim as optim    # 최적화 알고리즘 모듈

##############################################
# 1. 데이터 다운로드 및 전처리
##############################################
# S&P500 상위 20종목 티커 리스트 (예시)
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "TSLA", "NVDA", "BRK-B",
           "JNJ", "UNH", "V", "MA", "HD", "PG", "JPM", "BAC", "VZ", "DIS", "PFE", "XOM"]

# 현재 날짜와 10년 전 날짜를 계산하여 데이터 기간 설정
end_date = datetime.datetime.now().strftime('%Y-%m-%d')  # 현재 날짜 (종료일)
start_date = (datetime.datetime.now() - datetime.timedelta(days=365*10)).strftime('%Y-%m-%d')  # 10년 전 날짜 (시작일)

# yfinance를 사용하여 상위 20종목의 Adjusted Close 가격 데이터를 다운로드
data = yf.download(tickers, start=start_date, end=end_date)["Adj Close"]
data.dropna(inplace=True)  # 결측치가 포함된 행 제거

# 일별 수익률 계산: 각 날짜별 종가 변화율을 계산하여 수익률 데이터 생성
returns_df = data.pct_change().dropna()

##############################################
# 2. 벤치마크: 시가총액 비중 배분 계산
##############################################
# 각 종목의 시가총액 정보를 가져와 벤치마크 포트폴리오의 가중치(비중)를 계산
market_caps = []
for ticker in tickers:
    try:
        info = yf.Ticker(ticker).info  # 해당 종목의 정보 조회
        cap = info.get("marketCap", None)  # 시가총액 정보 추출
        # 만약 시가총액 정보가 없으면 기본값(예: 10억)을 할당
        if cap is None:
            cap = 1e9
        market_caps.append(cap)
    except Exception as e:
        # 정보 조회 중 오류가 발생하면 기본값 할당
        market_caps.append(1e9)

# 시가총액 배열을 numpy 배열로 변환하고, 전체 시가총액에 대한 각 종목의 비중 계산
market_caps = np.array(market_caps)
benchmark_weights = market_caps / market_caps.sum()

print("Benchmark weights (시가총액 비중):")
print(dict(zip(tickers, np.round(benchmark_weights, 4))))

# 각 종목의 일별 수익률에 시가총액 비중을 곱하여 벤치마크의 일별 수익률 계산
benchmark_daily_returns = returns_df.dot(benchmark_weights)

##############################################
# 3. 환경(Environment) 정의: HistoricalPortfolioEnv
##############################################
# Gym의 Env 클래스를 상속받아, 과거 수익률 데이터를 순차적으로 제공하는 포트폴리오 환경을 정의
class HistoricalPortfolioEnv(gym.Env):
    """
    과거 일별 수익률 데이터를 순차적으로 제공하는 환경.
    - 상태(state): 해당일의 각 종목의 수익률 (벡터 형태)
    - 행동(action): 각 종목에 할당할 투자 비중 (총합=1이어야 함)
    - 보상(reward): 선택한 투자 비중과 실제 수익률의 내적 (해당일 포트폴리오 수익률)
    """
    def __init__(self, returns):
        super(HistoricalPortfolioEnv, self).__init__()
        self.returns = returns            # numpy 배열 형태의 수익률 데이터, shape=(T, n_assets)
        self.n_assets = returns.shape[1]    # 자산(종목) 개수
        self.current_step = 0               # 데이터의 현재 시간 스텝
        # 행동 공간: 각 종목에 대한 투자 비중, 0~1 범위의 연속값 (벡터)
        self.action_space = spaces.Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)
        # 관측 공간: 각 종목의 수익률 (실수 값으로 무한대 범위)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_assets,), dtype=np.float32)

    def reset(self):
        # 환경 초기화: 시간 스텝을 0으로 초기화 후 첫 번째 수익률 벡터 반환
        self.current_step = 0
        return self.returns[self.current_step]

    def step(self, action):
        # 행동값을 정규화하여 총합이 1이 되도록 함
        weights = action / (np.sum(action) + 1e-8)
        # 현재 스텝의 종목별 수익률 가져오기
        current_return = self.returns[self.current_step]
        # 포트폴리오 수익률: 선택한 투자 비중과 실제 수익률의 내적 계산
        reward = np.dot(weights, current_return)
        self.current_step += 1  # 다음 시간 스텝으로 이동
        done = self.current_step >= len(self.returns)  # 마지막 스텝인지 확인
        if not done:
            next_state = self.returns[self.current_step]
        else:
            next_state = np.zeros(self.n_assets)  # 에피소드 종료 시 더 이상 관측 불가
        return next_state, reward, done, {}

# 수익률 데이터를 numpy 배열로 변환 후 환경 생성
returns_np = returns_df.values
env = HistoricalPortfolioEnv(returns_np)
n_assets = env.n_assets  # 환경 내 자산(종목) 수

##############################################
# 4. Actor-Critic 모델 정의 (PyTorch)
##############################################
# Actor-Critic 네트워크 정의: 정책(Actor)와 가치(Critic)를 동시에 학습
class ActorCritic(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ActorCritic, self).__init__()
        # 입력층에서 은닉층1로 연결하는 선형층
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        # 은닉층1에서 은닉층2로 연결하는 선형층
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # Actor 출력층: Dirichlet 분포의 알파 파라미터를 예측 (양의 값이어야 하므로 softplus 후 +1)
        self.actor = nn.Linear(hidden_dim, output_dim)
        # Critic 출력층: 상태의 가치를 예측하는 선형층 (스칼라 출력)
        self.critic = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        # 입력 데이터를 은닉층을 통과시키며 ReLU 활성화 함수 적용
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # Actor와 Critic의 결과값 계산
        actor_logits = self.actor(x)  # 정책 네트워크의 출력
        value = self.critic(x)          # 가치 함수의 출력
        return actor_logits, value

# 모델 파라미터 설정
input_dim = n_assets    # 입력 차원: 각 종목의 수익률 (자산 개수)
hidden_dim = 64         # 은닉층 차원
output_dim = n_assets   # 출력 차원: 각 종목에 대한 투자 비중 예측

# Actor-Critic 모델 인스턴스 생성 및 Adam 옵티마이저 설정
model = ActorCritic(input_dim, hidden_dim, output_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
gamma = 0.99  # 미래 보상 감가율 (discount factor)

##############################################
# 5. 학습 (Training) - Actor-Critic 방식
##############################################
num_episodes = 50  # 총 에피소드 수 (데모 목적)

for episode in range(num_episodes):
    state = env.reset()    # 환경 초기화 및 첫 상태 획득
    log_probs = []         # 각 스텝의 로그 확률 저장 리스트 (Actor 업데이트용)
    values = []            # Critic의 상태 가치 예측 저장 리스트
    rewards = []           # 에피소드 동안의 보상 저장 리스트
    done = False           # 에피소드 종료 여부 플래그

    while not done:
        # 현재 상태를 파이토치 텐서로 변환 (배치 차원 추가)
        state_tensor = torch.FloatTensor(state).unsqueeze(0)  # shape: [1, n_assets]
        # 모델을 통해 정책(Actor)와 가치(Critic)를 계산
        logits, value = model(state_tensor)
        # Dirichlet 분포의 알파 파라미터 계산: softplus 함수를 사용해 음수가 없도록 보정 후 +1
        alpha = F.softplus(logits) + 1.0
        # Dirichlet 분포 생성 (1차원 벡터 형태)
        dist = torch.distributions.Dirichlet(alpha.view(-1))
        # 분포에서 샘플링하여 행동(투자 비중) 결정
        action = dist.sample()
        # 선택된 행동의 로그 확률 계산 (학습 시 정책 기울기 업데이트에 사용)
        log_prob = dist.log_prob(action)
        # 환경에 행동 적용 후 다음 상태, 보상, 종료 여부 획득
        next_state, reward, done, _ = env.step(action.detach().numpy())

        # 각 변수들 기록
        log_probs.append(log_prob)
        values.append(value.squeeze(0))
        rewards.append(reward)
        state = next_state  # 상태 업데이트

    # 에피소드의 누적 보상(리턴)을 감가율(gamma)을 고려해 계산 (역순 계산)
    returns_episode = []
    R = 0
    for r in rewards[::-1]:
        R = r + gamma * R
        returns_episode.insert(0, R)
    returns_episode = torch.tensor(returns_episode, dtype=torch.float32)
    values = torch.stack(values)
    log_probs = torch.stack(log_probs)

    # Advantage 계산: 실제 누적 보상과 Critic의 예측 가치 차이
    advantages = returns_episode - values.detach()
    # Actor 손실: 로그 확률에 Advantage를 곱해 부호 반전 후 평균 (정책 경사법)
    actor_loss = -(log_probs * advantages).mean()
    # Critic 손실: 가치 예측과 실제 누적 보상의 평균 제곱 오차(MSE)
    critic_loss = F.mse_loss(values, returns_episode)
    # 최종 손실: Actor 손실과 Critic 손실의 합
    loss = actor_loss + critic_loss

    # 역전파 및 파라미터 업데이트
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 10 에피소드마다 학습 상태(손실, 총 보상) 출력
    if episode % 10 == 0:
        total_reward = sum(rewards)
        print(f"Episode {episode:3d} | Loss: {loss.item():.4f} | Total Reward: {total_reward:.4f}")

print("학습 완료")

##############################################
# 6. 백테스팅 (Backtesting)
##############################################
# 학습된 모델을 사용하여 전체 기간에 대해 포트폴리오 수익률 산출

# 백테스트를 위한 새로운 환경 인스턴스 생성
env_bt = HistoricalPortfolioEnv(returns_np)
state = env_bt.reset()
agent_daily_returns = []  # 에이전트가 생성한 일별 수익률 기록 리스트

while True:
    # 현재 상태를 텐서로 변환하여 모델 입력으로 사용
    state_tensor = torch.FloatTensor(state).unsqueeze(0)
    with torch.no_grad():
        # 모델을 통해 정책 네트워크의 출력 계산 (액션 결정)
        logits, _ = model(state_tensor)
        # Dirichlet 분포 파라미터 계산 (softplus 후 +1)
        alpha = F.softplus(logits) + 1.0
        dist = torch.distributions.Dirichlet(alpha.view(-1))
        # 백테스트에서는 확률 분포의 평균값을 사용하여 결정론적 정책으로 행동 결정
        weights = dist.mean
    # 선택한 투자 비중을 환경에 적용하여 다음 상태와 보상 획득
    next_state, reward, done, _ = env_bt.step(weights.numpy())
    agent_daily_returns.append(reward)
    if done:
        break
    state = next_state

# 에이전트의 일별 수익률을 numpy 배열로 변환 후 누적 수익률 계산 (초기 투자금 1 기준)
agent_daily_returns = np.array(agent_daily_returns)
agent_cumulative = np.cumprod(1 + agent_daily_returns)

# 벤치마크 누적 수익률 계산: 벤치마크 일별 수익률을 같은 기간에 맞추어 계산
benchmark_daily = benchmark_daily_returns.values[-len(agent_daily_returns):]
benchmark_cumulative = np.cumprod(1 + benchmark_daily)

# 백테스트 기간의 날짜 (원본 수익률 데이터의 인덱스 사용)
dates = returns_df.index[-len(agent_daily_returns):]

##############################################
# 7. 결과 Plot: RL 에이전트 vs. Benchmark
##############################################
# 백테스팅 결과를 시각화하여 RL 에이전트와 벤치마크(시가총액 비중)의 누적 수익률 비교
plt.figure(figsize=(12,6))
plt.plot(dates, agent_cumulative, label="RL Agent Portfolio")
plt.plot(dates, benchmark_cumulative, label="Benchmark (시가총액 비중)", linestyle="--")
plt.xlabel("날짜")
plt.ylabel("누적 수익률")
plt.title("Backtesting: RL Agent vs. Benchmark")
plt.legend()
plt.grid(True)
plt.show()
